<a href="https://colab.research.google.com/github/jmailman/AI_regulations_and_ethics_navigator_-AIREN-/blob/main/AI_regulations_and_ethics_navigator_(AIREN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI regulations and ethics navigator (AIREN)**

Joshua Banks Mailman, March 2024


This is a prototype retrieval based chatbot that focuses on AI ethics and regulation, in two senses:

(1) It enables developers to makes queries about current or potential AI capabilities and receive responses that are informed by scholarly documents on the ethics and morality of AI as well as the November 2023 White House Executive Order on AI and the March 2024 EU AI Act.

(2) It enables one to directly compare the White House Executive Order on AI and the EU AI Act with respect to any question, such as particular features to be added to an AI system

## Tools/packages used

`LangChain`

`ChromaDB`

`OpenAI`

The original documents as well as the vector store embeddings are kept on a Google drive.

# Setup

In [126]:
try:
  import pypdf
except:
  !pip install pypdf
  import pypdf

In [127]:
try:
  import chromadb
except:
  !pip install chromadb
  import chromadb

In [128]:
try:
  import tiktoken
except:
  !pip install tiktoken
  import tiktoken

In [129]:
try:
  import langchain
except:
  !pip install langchain
  import langchain

In [130]:

  #!pip install openai langchain -q
  !pip install -U langchain-openai


### Mount Google drive for persistence

In [9]:
from google.colab import drive
try:
  drive.flush_and_unmount()
except:
  pass

drive.mount('/content/gdrive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive


#### Folder structures

In [10]:
doc_folder = 'documents'
emb_folder = 'embeddings'

In [11]:
us_docs = 'us_regulations_etc'
eu_docs = 'eu_regulations_etc'

In [12]:
gdrive_data_folder = '/content/gdrive/My Drive/ai_regulations_and_ethics_navigation'
gdrive_data_docs = gdrive_data_folder + '/' + doc_folder
gdrive_data_embs = gdrive_data_folder + '/' + emb_folder


#### Curated list of relevant articles from the _Stanford Encyclopedia of Philosophy_

In [13]:
selected_sep_articles = [
'logic-ai',
'ethics-ai',
'artificial-intelligence',
'ethics-it-phenomenology',
'it-privacy',
'reasoning-automated',
'ethics-business',
'ethics-search',
'ethics-social-networking',
'ethics-manipulation',
'it-moral-values',
'computing-responsibility']


# privacy-medicine/
# computational-linguistics/
# computational-philosophy/
# technology/
# chimeras/
# mind-identity/
# turing-test/
# consciousness-animal/

In [14]:
from langchain.document_loaders import BSHTMLLoader, DirectoryLoader
subfolder = 'stanford_encyclopedia_of_philosophy'
gdrive_data_docs_subdir = gdrive_data_docs + '/' + subfolder

bshtml_dir_loader = DirectoryLoader(gdrive_data_docs_subdir, loader_cls=BSHTMLLoader)
html_data = bshtml_dir_loader.load()

In [15]:
selected_html_data = \
 [article for article in html_data if article.metadata['source'].split('/')[-1].replace('.html','').replace('_','-') in selected_sep_articles]
len(selected_html_data)

10

### US White House Exec Order and EU AI Act pdf files

In [17]:
files = [
    {
        "name": "us_regulations",
        "path": gdrive_data_docs + '/' + us_docs + '/' + \
           "Executive Order on the Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence _ United States, White House, November 2023.pdf"
    },
    {
        "name": "eu_regulations",
        "path": gdrive_data_docs + '/' + eu_docs + '/' + \
          "EU_AI_act.pdf",
    }

]


In [18]:
src_list = [item['path'] for item in files]
src_dict = dict()
src_dict = {item.split('/')[-1]:item for item in src_list }
src_dict

{'Executive Order on the Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence _ United States, White House, November 2023.pdf': '/content/gdrive/My Drive/ai_regulations_and_ethics_navigation/documents/us_regulations_etc/Executive Order on the Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence _ United States, White House, November 2023.pdf',
 'EU_AI_act.pdf': '/content/gdrive/My Drive/ai_regulations_and_ethics_navigation/documents/eu_regulations_etc/EU_AI_act.pdf'}

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pages = list()
for file in files:
    loader = PyPDFLoader(file["path"])
    pages += loader.load_and_split()

In [132]:
len(pages)

335

In [19]:
#! ls '/content/gdrive/My Drive/ai_regulations_and_ethics_navigation/documents/us_regulations_etc'
! ls '/content/gdrive/My Drive/ai_regulations_and_ethics_navigation/embeddings/'

### OpenAI credentials etc

In [20]:
import os
import getpass

OpenAI_token = getpass.getpass(prompt='Enter your OpenAI API token')
os.environ["OPENAI_API_KEY"] = OpenAI_token

Enter your OpenAI API token··········


# Populate the vector store

#### Combine pdf and html documents and split into chunks

In [23]:
all_texts = [*pages, *selected_html_data]
len(all_texts)

345

In [24]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=20)
chunked_documents = text_splitter.split_documents(pages)

In [25]:
len(chunked_documents)

335

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
rec_text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=20,separators = ["\n\n"])
chunked_html = rec_text_splitter.split_documents(selected_html_data)
len(chunked_html)

474

In [27]:
all_chunked_documents = [*chunked_documents, *chunked_html]
len(all_chunked_documents)

809

#### Set up the persistant ChromaDB vector store

In [28]:
persist_directory = gdrive_data_embs

if len(os.listdir( persist_directory )) > 0:  # Check if the directory is empty
    # Load existing ChromaDB collection
    vectordb = Chroma.load(persist_directory)


In [29]:
import os
from langchain.vectorstores import Chroma
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

persist_directory = gdrive_data_embs

if len(os.listdir( persist_directory )) > 0:  # Check if the directory is empty
    # Load existing ChromaDB collection
    vectordb = Chroma.load(persist_directory)
else:
    # Create a new ChromaDB collection
    vectordb = Chroma.from_documents(
        documents=all_chunked_documents,
        embedding=OpenAIEmbeddings(),
        persist_directory = persist_directory
    )


## Set up RAG pipeline

In [125]:
# In the future we might supply page numbers as references from the retrieval calls
# page_numbers = [doc.metadata['page'] for doc in result['source_documents']]
# page_numbers

[99, 18, 17, 106]

This function retrieves chunks that are only from the specified source. Two tools are associated with the different sources

In [30]:
from langchain.chains import RetrievalQA

def source_aware_retriever(vectorstore, source):
    def run(query):
        doc_retriever = vectordb.as_retriever(search_kwargs={"filter": {"source": source}})

        chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_retriever, return_source_documents=False)
        result = chain.invoke(query, metadata_filters={"source": source})
        #result = chain.invoke(query) # We already filtered it upstream so we don't need to do it again.
        return result
    return run

In [31]:
src = list(src_dict.keys())[0]
source_a_tool = source_aware_retriever(vectordb, source=src_dict[src])

src = list(src_dict.keys())[1]
source_b_tool = source_aware_retriever(vectordb, source=src_dict[src])

A general retriever that doesn't specify the source

In [94]:
def general_retriever(vectorstore):
    def run(query):
        doc_retriever = vectordb.as_retriever(search_kwargs={"k":2})

        chain = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=doc_retriever, return_source_documents=True)
        result = chain.invoke(query)
        return result
    return run

In [95]:
all_sources_tool = general_retriever(vectordb)

Determine whether the question is asking for a binary comparison (between US and EU regulations).
For now, it just looks for the keyword `compare` although this could be made more robust.

In [36]:
def classify_question(question):
    # ... Implement your classification logic here...
    is_comparative = False
    if 'compare' in question.lower():
      is_comparative = True
    return is_comparative

Tools derived from the functions defined above

In [80]:
from langchain.agents import Tool

classifier_tool = Tool(
        name = "Classify question",
        func=classify_question,
        description="useful for determining if a question is asking for a comparison."
    )
retriever_tool_source_a = Tool(
        name = "White House Executive Order, November 2023",
        func=source_a_tool,
        description="useful for determining what the White House Executive Order has to say on the topic."
    )
retriever_tool_source_b = Tool(
        name = "European Union AI Act, March 2024",
        func=source_b_tool,
        description="useful for determining what EU AI Act has to say on the topic."
    )
retriever_tool_all_sources = Tool(
        name = "White House Executive Order, EU AI Act, and Stanford Encyclopedia of Philosophy",
        func=all_sources_tool,
        description="useful for determining what rigorous sources have to say on the topic."
)

LLM setup, for text generation

In [99]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain import OpenAI, LLMChain, PromptTemplate

llm = OpenAI(temperature=0,  max_tokens=300)
llm_longer_response = OpenAI( temperature=0,  max_tokens=350)

Prompt templates

In [39]:
comparator_prompt_template = \
  PromptTemplate.from_template("""You have the following responses on the issue of '{issue}':
* {source_a_name}: {source_a_response}
* {source_b_name}: {source_b_response}
Analyze the differences in positions taken by the two sources.
Clearly state where they agree, where they disagree, and any important nuances in their stances.
(Aim for about 200 words, in completed sentences.)
""")

comparator_tool = LLMChain(llm=llm, prompt=comparator_prompt_template, verbose=True)

In [100]:
analysis_prompt_template = \
  PromptTemplate.from_template("""You have the following responses on the issue of '{issue}':
* {source_documents}
Analyze the various angles on the issue, with respect to ethics, morality, and regulatory consisderations.
Cite peoples' names (and publication dates) wherever possible.
(Aim for about 300 words, in completed sentences.)
""")

#analysis_tool = LLMChain(llm=llm_longer_response, prompt=analysis_prompt_template, verbose=True)
analysis_tool = LLMChain(llm=llm_longer_response, prompt=analysis_prompt_template, verbose=True)

In [114]:
sentence_completion_prompt_template = \
  PromptTemplate.from_template("""Based on the preceding context '{context}', provide only the remainder of
  the final incomplete sentence. Do not include any of the context already provided.
  Return only the new part that the completes the sentence.""")
sentence_completion_tool = LLMChain(llm=llm, prompt=sentence_completion_prompt_template, verbose=False)

### The main event:  This function `run_agent` determines the flow of response generation

• If the query is classified as a _comparison_ then it's going to retrieve chunks from the WH Exec Order and EU AI Act.
Then a `comparator_tool` takes those chunks and synthesizes a response.

• Otherwise all the sources, including a curated list of articles from the _Stanford Encyclopedia of Philosophy_, are queried.
From the chunks returned, an response is synthesized.

Either way, the summary returned often truncates in mid sentence, so a completion tool takes the last 7 complete sentences and generates a sentence completion for the final incomplete sentence and appends it to the original summary.



In [124]:
def run_agent(query):
    is_comparative = classifier_tool.run(query)
    if is_comparative:
        source_a_response = retriever_tool_source_a.invoke(query)
        source_b_response = retriever_tool_source_b.invoke(query)

        # Enhanced with {issue} extraction (if needed)
        #issue = extract_issue_from_query(query)
        issue = query

        summary = comparator_tool.run(
            source_a_name = retriever_tool_source_a.name,
            source_a_response=source_a_response,

            source_b_name = retriever_tool_source_b.name,
            source_b_response=source_b_response,

            issue=issue
        )
        #return summary
    else:
        # ... handle non-comparative queries ...
        issue = query
        all_sources_response = retriever_tool_all_sources(query)
        print(len(all_sources_response))
        summary = analysis_tool.run(
          source_documents = all_sources_response,
          issue=issue
        )
    if summary[-1] == '.':
      return summary
    else:
      context = '. '.join( summary.split('.')[-7:])
      completion = ' ' + sentence_completion_tool.run(context).strip('\n\n').strip('\'')
      return summary + " " + completion
#

In [115]:
summary = """In addition to ethical considerations, there are also moral implications to be considered. The use of AI systems to manipulate behavior can have a significant impact on individuals and society as a whole. As mentioned by Woolley and Howard (2017), the use of social media for political propaganda can influence voting behavior and potentially harm the autonomy of individuals. This raises questions about the moral responsibility of companies and governments in using AI systems for manipulative purposes.
From a regulatory perspective, there are also several considerations to be taken into account. As mentioned by Schneier (2015), the business model of the internet is based on surveillance and the collection of personal data. This raises concerns about the protection of privacy and the ownership of data. Wachter and"""

context = '. '.join( summary.split('.')[-7:])
completion = ' ' + sentence_completion_tool.run(context).strip('\n\n').strip('\'')
summary + " " + completion

'In addition to ethical considerations, there are also moral implications to be considered. The use of AI systems to manipulate behavior can have a significant impact on individuals and society as a whole. As mentioned by Woolley and Howard (2017), the use of social media for political propaganda can influence voting behavior and potentially harm the autonomy of individuals. This raises questions about the moral responsibility of companies and governments in using AI systems for manipulative purposes.\nFrom a regulatory perspective, there are also several considerations to be taken into account. As mentioned by Schneier (2015), the business model of the internet is based on surveillance and the collection of personal data. This raises concerns about the protection of privacy and the ownership of data. Wachter and  Mittelstadt (2018) argue that there is a need for transparency and accountability in the use of AI systems, as well as clear guidelines for ethical and responsible use.'

## Demonstrate RAG pipeline

In [49]:
from IPython.display import display, Markdown

In [120]:
response = run_agent("We might add a feature to our AI system that some say manipulates behavior. What should we consider?")
display(Markdown( response ))

3


> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'We might add a feature to our AI system that some say manipulates behavior. What should we consider?':
* {'query': 'We might add a feature to our AI system that some say manipulates behavior. What should we consider?', 'result': ' The ethical implications and potential consequences of manipulating behavior through AI should be carefully considered.', 'source_documents': [Document(page_content='The ethical issues of AI in surveillance go beyond the mere\naccumulation of data and direction of attention: They include\nthe use of information to manipulate behaviour, online and\noffline, in a way that undermines autonomous rational choice. Of\ncourse, efforts to manipulate behaviour are ancient, but they may gain\na new quality when they use AI systems. Given users’ intense\ninteraction with data systems and the deep knowledge about individuals\nthis provides, they are vulnerable 


The issue of adding a feature to an AI system that manipulates behavior raises several ethical, moral, and regulatory considerations. One of the main ethical concerns is the potential consequences of manipulating behavior through AI. As mentioned in the source documents, this can undermine autonomous rational choice and harm the autonomy of individuals (Susser, Roessler, and Nissenbaum, 2019). This raises questions about the ethical responsibility of those developing and implementing such a feature, as well as the potential harm it could cause to individuals and society as a whole.

From a moral perspective, the use of AI to manipulate behavior raises questions about the intentions and motivations behind such actions. As mentioned in the source documents, many advertisers and marketers already use manipulation and deception to maximize profit (Costa and Halpern, 2019). Adding AI to this equation could potentially amplify these unethical practices and further exploit human weaknesses. This raises moral concerns about the impact on individuals and society, as well as the responsibility of those involved in the development and use of such technology.

In terms of regulatory considerations, the use of AI to manipulate behavior raises questions about privacy and data protection. As mentioned in the source documents, AI systems rely on vast amounts of data for training, which can often involve a trade-off between privacy and technical quality (Wachter and Mittelstadt, 2019). This highlights the need for regulations that protect individuals' privacy and rights to their data, while also ensuring the quality and safety of AI systems.

Furthermore, the use of AI to manipulate behavior also raises concerns about the potential for discrimination and bias. As AI systems are trained on existing data, they can perpetuate and amplify existing biases and discrimination (Burr and Christianini, 2019  , which can have harmful consequences for marginalized groups and perpetuate systemic inequalities. This highlights the need for regulations that address and mitigate bias in AI systems, as well as promote diversity and inclusivity in the development and use of such technology.

In [121]:
response = run_agent("Are there different AI bias considerations to worry about in the US and Europe?")
display(Markdown( response ))

3


> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'Are there different AI bias considerations to worry about in the US and Europe?':
* {'query': 'Are there different AI bias considerations to worry about in the US and Europe?', 'result': ' Yes, there are different AI bias considerations in the US and Europe. The EU has stricter privacy regulations, while the US and China prioritize growth with less regulation. The political dimensions of automated systems in the US are also being investigated.', 'source_documents': [Document(page_content='The policy in this field has its ups and downs: Civil liberties and\nthe protection of individual rights are under intense pressure from\nbusinesses’ lobbying, secret services, and other state agencies\nthat depend on surveillance. Privacy protection has diminished\nmassively compared to the pre-digital age when communication was based\non letters, analogue telephone communications, and pers


The issue of AI bias considerations in the US and Europe raises various ethical, moral, and regulatory concerns. One of the main ethical concerns is the potential for discrimination and unfair treatment of individuals based on their race, gender, or other characteristics. This is a concern in both the US and Europe, as evidenced by the examples of biased AI systems in both regions. In the US, the COMPAS system was found to be biased against black defendants, while in Europe, the EU General Data Protection Regulation was implemented to address privacy concerns and potential discrimination.

From a moral perspective, the use of AI systems that perpetuate bias and discrimination raises questions about the values and principles that underpin these systems. As mentioned in the source documents, the human cognitive system is prone to various biases, and there is a question of whether AI systems should also have these biases. This raises moral questions about the responsibility of developers and policymakers to ensure that AI systems are designed and used in an ethical and fair manner.

In terms of regulatory considerations, there are differences between the US and Europe. The EU has stricter privacy regulations, while the US and China prioritize growth with less regulation. This highlights the different approaches to AI regulation in these regions. In the US, there is a focus on promoting innovation and economic growth, while in Europe, there is a greater emphasis on protecting individual rights and privacy. This raises questions about the role of government in regulating AI and the balance between promoting innovation and protecting individuals from potential harm.

Furthermore, the political dimensions of automated systems in the US are also being investigated, as mentioned in the source documents. This highlights the need for regulatory frameworks that not only address technical aspects of AI but also consider the societal and political implications of these systems.

In conclusion  the need for comprehensive and thoughtful regulation of AI that takes into account both technical and societal factors.

In [102]:
response = run_agent("We might add a feature to our AI system that some say manipulates behavior. What should we consider?")
display(Markdown( response ))

3


> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'We might add a feature to our AI system that some say manipulates behavior. What should we consider?':
* {'query': 'We might add a feature to our AI system that some say manipulates behavior. What should we consider?', 'result': ' The ethical implications and potential consequences of manipulating behavior through AI should be carefully considered.', 'source_documents': [Document(page_content='The ethical issues of AI in surveillance go beyond the mere\naccumulation of data and direction of attention: They include\nthe use of information to manipulate behaviour, online and\noffline, in a way that undermines autonomous rational choice. Of\ncourse, efforts to manipulate behaviour are ancient, but they may gain\na new quality when they use AI systems. Given users’ intense\ninteraction with data systems and the deep knowledge about individuals\nthis provides, they are vulnerable 


The issue of adding a feature to an AI system that manipulates behavior raises several ethical, moral, and regulatory considerations. One of the main ethical concerns is the potential consequences of manipulating behavior through AI. As mentioned in the source documents, this can undermine autonomous rational choice and harm the autonomy of individuals (Susser, Roessler, and Nissenbaum, 2019). This raises questions about the ethical responsibility of those developing and implementing such a feature, as well as the potential harm it could cause to individuals and society as a whole.

From a moral perspective, the use of AI to manipulate behavior raises questions about the intentions and motivations behind such actions. As mentioned in the source documents, many advertisers and marketers already use manipulation and deception to maximize profit (Costa and Halpern, 2019). Adding AI to this equation could potentially amplify these unethical practices and further exploit human weaknesses. This raises moral concerns about the impact on individuals and society, as well as the responsibility of those involved in the development and use of such technology.

In terms of regulatory considerations, the use of AI to manipulate behavior raises questions about privacy and data protection. As mentioned in the source documents, AI systems rely on vast amounts of data for training, which can often involve a trade-off between privacy and technical quality (Wachter and Mittelstadt, 2019). This highlights the need for regulations that protect individuals' privacy and rights to their data, while also ensuring the quality and safety of AI systems.

Furthermore, the use of AI to manipulate behavior also raises concerns about the potential for discrimination and bias. As AI systems are trained on existing data, they can perpetuate and amplify existing biases and discrimination (Burr and Christianini, 2019

In [50]:
response = run_agent("What are the issues the might be considered if we add 'device fingerprinting' to our AI system?")
display(Markdown( response ))


The addition of 'device fingerprinting' to AI systems raises ethical concerns about privacy, profiling, manipulation, and loss of control over personal data. These issues have already been amplified by other AI technologies, such as face recognition and surveillance, and may continue to be a source of controversy and public attention. Additionally, the use of AI in device fingerprinting can further exacerbate issues of opacity and bias in decision-making processes. Therefore, it is crucial for policy makers to carefully consider and regulate the use of device fingerprinting in AI systems, as well as ensure responsible design to protect privacy and address potential biases.

One of the main ethical concerns surrounding the use of device fingerprinting in AI systems is the violation of privacy. As mentioned in the source documents, the digital world has made it much harder to control who collects and has access to personal data. This raises questions about the ownership and control of personal data, as well as the potential for manipulation and exploitation by businesses and state agencies. This issue has been further complicated by the use of AI in device fingerprinting, which can collect and analyze vast amounts of data without the knowledge or consent of individuals.

Another ethical consideration is the potential for bias in decision-making processes. AI systems rely on vast amounts of data to make decisions, and this data may contain inherent biases. This can lead to discriminatory outcomes, particularly in areas such as employment and criminal justice. The use of device fingerprinting in AI systems can exacerbate this issue, as it may rely on data that is already biased due to factors such as socioeconomic status or race.

From a moral standpoint, the use of device fingerprinting in AI systems raises questions about the autonomy and agency of individuals. As AI systems become more advanced and capable of making decisions on behalf of humans, there is a concern that individuals may lose control over their own lives. This is particularly relevant in the context of device fingerprinting, as it can track and analyze personal data without the knowledge or consent of individuals.

In terms of regulatory considerations, there is a need for policies and laws to address the ethical concerns surrounding device fingerprinting in AI systems. This includes regulations on data collection and usage, as well as guidelines for responsible design and development of AI systems. Additionally, there is a need for transparency and accountability in the use of AI, particularly in regards to decision-making processes that may have significant impacts on individuals and society as a whole.

In conclusion, the addition of device fingerprinting to AI systems raises a range of ethical,

In [83]:
response = run_agent("What should we consider as, or if, we try to program morality into autonomous weapons systems or vehicles?")
display(Markdown( response ))

3


> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'What should we consider as, or if, we try to program morality into autonomous weapons systems or vehicles?':
* {'query': 'What should we consider as, or if, we try to program morality into autonomous weapons systems or vehicles?', 'result': ' When considering programming morality into autonomous weapons systems or vehicles, we should consider questions on how they should behave and how responsibility and risk should be distributed in the complicated system they operate in.', 'source_documents': [Document(page_content='In the absence of any definitive arguments for or against the\npossibility of future computer systems being morally responsible,\nresearchers within the field of machine ethics aim to further develop\nthe discussion by focusing instead on creating computer system that\ncan behave as if they are moral agents (Moor 2006,\nCervantes et al 2019 , Zoshak and Dew 2021


The issue of programming morality into autonomous weapons systems or vehicles raises a number of ethical, moral, and regulatory considerations. One key question is how these systems should behave and make decisions in morally complex situations. This has been a topic of discussion in the field of machine ethics, with researchers aiming to create computer systems that can behave as if they are moral agents (Moor, 2006; Cervantes et al., 2019; Zoshak & Dew, 2021). This involves designing systems that can independently determine the right course of action in a given situation, taking into account the moral and social implications of their behavior.

Another important consideration is the distribution of responsibility and risk in these systems. As autonomous technologies become more complex and capable of operating without direct human control, there is a need to determine who is responsible for the actions of these systems. Allen and Wallach (2012) argue that autonomous moral agents (AMAs) must be able to reason about the moral and social significance of their behavior and use this information to make appropriate choices. This is particularly relevant in contexts such as military robots, driverless cars, and service robots, where the decisions made by these systems can directly impact the safety and well-being of humans.

In terms of regulatory considerations, there is a need to adapt existing laws and standards to accommodate autonomous systems. This includes civil and criminal liability, as well as technical standards for safety and security. Bodies like the IEEE and BSI have already produced standards

# Compare US and EU regulations
#### Use the keyword `compare` to signal a question concering comparing the White House Executive order versus the EU AI Act

In [125]:
response = run_agent("Compare the US and European stances on education with regard to AI technology")
display(Markdown( response ))



> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'Compare the US and European stances on education with regard to AI technology':
* White House Executive Order, November 2023: {'query': 'Compare the US and European stances on education with regard to AI technology', 'result': " The US and European stances on education with regard to AI technology differ in some ways. The US has a more proactive approach, with a focus on promoting AI education and training in schools and universities, as well as investing in research and development. The European approach is more cautious, with a focus on regulating and managing the risks of AI, including potential job displacement and discrimination. Both regions recognize the importance of educating the public about AI and its potential impact, but the US tends to prioritize promoting innovation and competitiveness, while Europe prioritizes protecting citizens' rights and privacy."} 
* Europe


Both the White House Executive Order and the European Union AI Act recognize the importance of education in the development and use of AI technology. They both agree that educating the public about AI and its potential impact is crucial. However, they differ in their approaches and priorities.

The White House Executive Order highlights the US's proactive approach towards promoting AI education and training in schools and universities. It also emphasizes the importance of investing in research and development to maintain the US's competitiveness in the global AI market. In contrast, the European Union AI Act takes a more cautious approach, with a focus on regulating and managing the potential risks and harms of AI technology. This includes classifying certain AI systems used in education as high-risk and subjecting them to additional safeguards and oversight.

One key difference between the two sources is their emphasis on protecting citizens' rights and privacy. While the US prioritizes promoting innovation and competitiveness, the European Union places a strong emphasis on safeguarding against potential harms and discrimination. This is evident in the European Union AI Act's call for interdisciplinary cooperation between AI developers and experts on inequality and non-discrimination.

Another important nuance in their stances is their focus on socially and environmentally beneficial outcomes. The US Executive Order highlights the importance of using AI technology to address issues such as accessibility for persons with disabilities, socio-economic inequalities, and environmental targets. On the other hand, the European Union AI Act also emphasizes the need for responsible and ethical development and use of AI in education, but does not explicitly mention  the potential positive impacts on society and the environment.

In [123]:
response = run_agent("Compare the US and European stances on education with regard to AI technology")
display(Markdown( response ))



> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'Compare the US and European stances on education with regard to AI technology':
* White House Executive Order, November 2023: {'query': 'Compare the US and European stances on education with regard to AI technology', 'result': " The US and European stances on education with regard to AI technology differ in some ways. The US has a more proactive approach, with a focus on promoting AI education and training in schools and universities, as well as investing in research and development. The European approach is more cautious, with a focus on regulating and managing the risks of AI, including potential job displacement and discrimination. Both regions recognize the importance of educating the public about AI and its potential impact, but the US tends to prioritize promoting innovation and competitiveness, while Europe prioritizes protecting citizens' rights and privacy."} 
* Europe


Both the White House Executive Order and the European Union AI Act recognize the importance of education in the development and use of AI technology. They both acknowledge the potential benefits of AI in education, such as promoting innovation and addressing societal issues. However, they differ in their approaches to achieving these goals.

The White House Executive Order takes a more proactive stance, with a focus on promoting AI education and training in schools and universities, as well as investing in research and development. This approach prioritizes the advancement of AI technology and its potential impact on the economy and global competitiveness. In contrast, the European Union AI Act takes a more cautious approach, with a focus on regulating and managing the risks of AI, including potential job displacement and discrimination. This approach prioritizes protecting individuals' rights and privacy.

One key difference between the two sources is their emphasis on regulation. While the US approach focuses on promoting innovation and competitiveness, the EU approach prioritizes protecting citizens' rights and privacy. This is reflected in the EU's efforts to develop a comprehensive regulatory framework for AI systems, including those used in education. The EU also classifies certain AI systems used in education as high-risk and implements additional safeguards to protect against discrimination and violations of privacy and data protection laws.

Another difference is the level of interdisciplinary cooperation encouraged by the EU. The EU emphasizes the importance of collaboration between AI developers and experts in areas such as non-discrimination, accessibility, and digital rights. This highlights the EU's focus on addressing potential ethical and societal implications

In [110]:
response = run_agent("Compare regulations on CBRN weapons issues in AI")
display(Markdown( response ))



> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'Compare regulations on CBRN weapons issues in AI':
* White House Executive Order, November 2023: {'query': 'Compare regulations on CBRN weapons issues in AI', 'result': ' The regulations on CBRN weapons issues in AI include evaluating the potential for AI to be misused to enable the development or production of CBRN threats, while also considering the benefits and application of AI to counter these threats. Additionally, there are requirements for regulating or overseeing the training, deployment, publication, or use of AI models that may present CBRN risks to the United States, and recommendations for minimizing the risks of AI model misuse. There are also guidelines for performing security reviews and managing potential security risks of releasing federal data that could aid in the development of CBRN weapons.'} 
* European Union AI Act, March 2024: {'query': 'Compare regulat


Both the White House Executive Order and the European Union AI Act address the issue of regulating CBRN weapons in the context of AI. However, there are some key differences in their positions.

Firstly, both sources agree that there is a need to evaluate the potential for AI to be misused in the development or production of CBRN weapons. They also both recognize the potential benefits of AI in countering these threats. This shows a shared understanding of the potential risks and benefits of AI in the context of CBRN weapons.

However, the White House Executive Order takes a more proactive approach by requiring regulation and oversight of the training, deployment, publication, and use of AI models that may present CBRN risks. This indicates a stronger stance on the need for strict control and monitoring of AI in order to prevent its misuse in the development of CBRN weapons.

On the other hand, the European Union AI Act does not specifically mention CBRN weapons in its context. This could suggest a less urgent or comprehensive approach to regulating AI in the context of CBRN weapons. However, it is important to note that the Act does address the broader issue of AI regulation and ethics, which could indirectly impact the development and use of AI in relation to CBRN weapons.

Overall, while both sources recognize the potential risks and benefits of AI in the context of CBRN weapons, the White House Executive Order takes a more specific and proactive stance on regulating and monitoring AI, while the

In [73]:
#run_agent("Compare regulations on synthetic nucleic acid issues in AI")
run_agent("Compare regulations on CBRN weapons issues in AI")



> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'Compare regulations on CBRN weapons issues in AI':
* White House Executive Order, November 2023: {'query': 'Compare regulations on CBRN weapons issues in AI', 'result': ' The regulations on CBRN weapons issues in AI include evaluating the potential for AI to be misused to enable the development or production of CBRN threats, while also considering the benefits and application of AI to counter these threats. Additionally, there are requirements for regulating or overseeing the training, deployment, publication, or use of AI models that may present CBRN risks to the United States, and recommendations for minimizing the risks of AI model misuse. There are also guidelines for performing security reviews and managing potential security risks of releasing federal data that could aid in the development of CBRN weapons.'} 
* European Union AI Act, March 2024: {'query': 'Compare regulat

'\nBoth the White House Executive Order and the European Union AI Act address the issue of regulating CBRN weapons in the context of AI. However, there are some key differences in their approaches and stances.\n\nOne area of agreement is the recognition of the potential for AI to be misused in the development or production of CBRN weapons. Both sources acknowledge the need for regulations to address this risk and mitigate it. They also both emphasize the importance of considering the benefits and applications of AI in countering CBRN threats.\n\nHowever, there are also notable differences in their positions. The White House Executive Order takes a more proactive approach, with specific requirements for regulating the training, deployment, publication, and use of AI models that may pose CBRN risks. It also includes recommendations for minimizing the risks of AI model misuse. On the other hand, the European Union AI Act does not mention CBRN weapons specifically and does not have any spe

In [74]:
run_agent("Compare regulations on CBRN weapons issues in AI")



> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'Compare regulations on CBRN weapons issues in AI':
* White House Executive Order, November 2023: {'query': 'Compare regulations on CBRN weapons issues in AI', 'result': ' The regulations on CBRN weapons issues in AI include evaluating the potential for AI to be misused to enable the development or production of CBRN threats, while also considering the benefits and application of AI to counter these threats. Additionally, there are requirements for regulating or overseeing the training, deployment, publication, or use of AI models that may present CBRN risks to the United States, and recommendations for minimizing the risks of AI model misuse. There are also guidelines for performing security reviews and managing potential security risks of releasing federal data that could aid in the development of CBRN weapons.'} 
* European Union AI Act, March 2024: {'query': 'Compare regulat

'\nBoth the White House Executive Order and the European Union AI Act address the issue of regulating CBRN weapons in the context of AI. However, there are some key differences in their approaches and stances.\n\nOne area of agreement is the recognition of the potential for AI to be misused in the development or production of CBRN weapons. Both sources acknowledge the need for regulations to address this risk and mitigate it. They also both emphasize the importance of considering the benefits and applications of AI in countering CBRN threats.\n\nHowever, there are also notable differences in their positions. The White House Executive Order takes a more proactive approach, with specific requirements for regulating the training, deployment, publication, and use of AI models that may pose CBRN risks. It also includes recommendations for minimizing the risks of AI model misuse. On the other hand, the European Union AI Act does not mention CBRN weapons specifically and does not have any spe

In [54]:
run_agent("Compare regulations on biometric data in AI")

'\nBoth the White House Executive Order and the European Union AI Act address the use of biometric data in AI and aim to protect individuals from potential harm and discrimination. They both recognize the importance of privacy and the need for transparency when using biometric data in AI systems.\n\nHowever, there are some differences in their positions. The White House Executive Order does not specifically mention regulations on biometric data in AI, but rather focuses on the need to protect people with disabilities from unequal treatment and the use of privacy-enhancing technologies. On the other hand, the European Union AI Act has specific regulations in place, such as prohibiting the use of biometric data to infer sensitive personal information and restricting its use for remote identification and social scoring.\n\nAnother difference is that the White House Executive Order is from November 2023, while the European Union AI Act is from March 2024. This could indicate that the Europ

In [59]:
run_agent("Compare regulations on copyright and creative works in AI")



> Entering new LLMChain chain...
Prompt after formatting:
You have the following responses on the issue of 'Compare regulations on copyright and creative works in AI':
* White House Executive Order, November 2023: {'query': 'Compare regulations on copyright and creative works in AI', 'result': ' The regulations on copyright and creative works in AI are addressed in section (c) of the Executive Order. This section requires the Under Secretary of Commerce for Intellectual Property and Director of the United States Patent and Trademark Office (USPTO Director) to publish guidance on inventorship and the use of AI in the inventive process. It also requires the USPTO Director to consult with the Director of the United States Copyright Office and issue recommendations on potential executive actions relating to copyright and AI. Additionally, the Secretary of Homeland Security is tasked with developing a training, analysis, and evaluation program to mitigate AI-related IP risks. These regula

'\nBoth the White House Executive Order and the European Union AI Act address the issue of copyright and creative works in AI. They both recognize the need for regulations to address potential issues such as inventorship, copyright protection, and discrimination in the use of AI in creative processes.\n\nOne key difference between the two sources is the approach to addressing these issues. The White House Executive Order focuses on guidance and recommendations, while the European Union AI Act includes specific requirements and obligations for providers of general-purpose AI models. The Executive Order tasks the USPTO Director with publishing guidance and consulting with the Director of the United States Copyright Office, while the AI Act requires providers to have a policy in place to respect Union copyright law and make a summary of the training data publicly available.\n\nAnother difference is the level of detail and specificity in the regulations. The Executive Order does not provid

In [ ]:
response = run_agent("Compare regulations on copyright and creative works in AI")
display(Markdown( response ))

Both the White House Executive Order and the European Union AI Act address the issue of copyright and creative works in AI. They both recognize the need for regulations to address potential issues such as inventorship, copyright protection, and discrimination in the use of AI in creative processes.

One key difference between the two sources is the approach to addressing these issues. The White House Executive Order focuses on guidance and recommendations, while the European Union AI Act includes specific requirements and obligations for providers of general-purpose AI models. The Executive Order tasks the USPTO Director with publishing guidance and consulting with the Director of the United States Copyright Office, while the AI Act requires providers to have a policy in place to respect Union copyright law and make a summary of the training data publicly available.

Another difference is the level of detail and specificity in the regulations. The Executive Order does not provide specific details on how to address these issues, while the AI Act includes specific requirements for providers and the AI Office to monitor compliance. The AI Act also takes into account the need to protect trade secrets and confidential business information, while the Executive Order does not mention this.

However, both sources agree on the importance of addressing these issues and recognize the potential risks and challenges posed by AI in the creative industry. They also both acknowledge the need